# Data Anaysis Project part1:

I am using Vancouver rental ads on craigslist to analyze rental market. Later, I will create the class to be able to analyze dynamically. Also, I will start collecting the data and save them to the CSV, so I can analyze the trend of the rental market as well. 

I will follow what I am learning from Google Data Analytics course to complete this project.

* Ask
* Prepare
* Process
* Analyze
* Share
* Act



## Overview of the project:

Analyze housing rental market in Fraser Valley to find out what is hot to real estate investors in this area. Iam going to create the code to analyze dynamically so the investors can apply in their area for their own use.  


## ASK 
    
    Find out what is 'HOT' in the market for investor, so i can find a property for them. Find out supply and demand of the rental housing market and calculate appropreate purchase price. 

#### What are the questions to ask?
Supply side:
* How much is the market rental price?
* What type of the rental is most popular and least popular?
* How many ads per day? Number of supply 
* Is rental price rising or decreasing over time?
* What is the correlation between housing price and rental price?
* From those questions, what is my prediction of the rental market? 

Demand side:
* What kind of propety does people look for rent?
* What is the price range they are looking for?
* What are the purposes of renting. (Who is looking for rent)





## Prepare  

#### Using Craigslist add to collect market data: (Supply side)
Scrape craigslist rental housing pages and store in csv file. 

<a href='https://github.com/rileypredum/East-Bay-Housing-Web-Scrape/blob/master/EB_Apt_Prices_Final.ipynb'>Code: I will use this code and modify for my situation.  </a> <br>
<a href='https://towardsdatascience.com/web-scraping-craigslist-a-complete-tutorial-c41cea4f4981'>Explaning</a>

## Process

I noticed that the all the values are string, so I will convert to correct data type as I collect the data. 

* sub_city_van_area: 
        * 'van'- Vancouver
        * 'bnc' - Burnaby/NewWest
        * 'rds' - Delta/Surrey/Langley
        * 'nvn' - North Shore
        * 'rch' - Richmond
        * 'pml' - Tricities/Pittmedow/Maple Ridge
* sub_city_tor_area: 
        * 'tor' - City of Toronto
        * 'bra' - Brampton
        * 'drh' - Durham Region
        * 'mss' - Mississauga
        * 'oak' - Oakville
        * 'yrk' - York Region

In [3]:
# Import libralis
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

from time import sleep
import re
from random import randint
from warnings import warn
from time import time
from IPython.display import clear_output

In [39]:
# Elliminate duplicate and the price listing less than $99.
def get_url(city_name, num_post, sub_city=None):
    if city_name == 'Abbotsford':
        return f'https://{city_name.lower()}.craigslist.org/search/apa?s={num_post}sort=date&bundleDuplicates=1&min_price=99max_price=&availabilityMode=0&sale_date=all+dates'
    if sub_city is not None:
        sub_city = sub_city + '/'
    return f'https://{city_name.lower()}.craigslist.org/search/{sub_city.lower()}apa?s={num_post}sort=date&bundleDuplicates=1&min_price=99max_price=&availabilityMode=0&sale_date=all+dates'
def get_num_posts()

In [45]:
city_names = ['Vancouver', 'Abbotsford', 'Toronto']
sub_cities_van_area = ['van', 'bnc', 'rds', 'nvn', 'rch', 'pml']
sub_cities_tor_area = ['tor', 'bra', 'drh', 'mss', 'oak', 'yrk']
urls = []

for city in city_names:
    if city == "Vancouver":
        city_name = city
        for sub_city in sub_cities_van_area:
            url = get_url(city_name=city_name )

'https://vancouver.craigslist.org/search/van/apa?s=0sort=date&bundleDuplicates=1&min_price=99max_price=&availabilityMode=0&sale_date=all+dates'

In [8]:
# Chilliwack rental housing

response = get(get_url(city_name = 'Abbotsford', num_post = 120))


soup = BeautifulSoup(response.text, 'html.parser')

posts = soup.find_all('li', class_='result-row')
print(type(posts)) # to double check that I got resultSet
print(len(posts)) # to double check I got 120 elements

<class 'bs4.element.ResultSet'>
120


In [5]:
%%time
# Find the total number of posts to find the limit of the pagination
city = 'Vancouver'
response = get(get_url(city_name = city, num_post=0))
soup = BeautifulSoup(response.text, 'html.parser')
posts = soup.find_all('li', class_='result-row')
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
print(f"Total number of search result: {results_total}")

# each page has 119 posts so each new page is defined as follows; s=120, s=240, .....
pages = np.arange(0, results_total+1, 120)

iterations = 0

post_date = []
post_cities = []
num_bedroom = []
sqfts = []
prices = []
post_titles = []
post_links = []

for page in pages:
    
    # get request
    response = get(get_url(city_name=city, num_post=page))
    
    sleep(1)
    
    # throw warning for status code that are not 200
    if response.status_code != 200:
        warn(f'Request: {requests}; Status code: {response.status_code}')
        
    soup = BeautifulSoup(response.text, 'html.parser')
    
    posts = soup.find_all('li', class_='result-row')
    
    
    # Extract data 
    for post in posts:
        
        if post.find('span', class_ = 'result-hood') is not None:
            
            # Posting date
            post_datetime = post.find('time', class_='result-date')['datetime']
            post_date.append(post_datetime)
            
            # Neighbourhoods
            post_city = post.find('span', class_='result-hood').text.strip('( )')
            post_cities.append(post_city)
            
            # title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_titles.append(post_title_text)
            
            # Post price in integer
            price = int(post.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
            prices.append(price)
            
            # Post link
            post_link = post_title['href']
            post_links.append(post_link)
            
            if post.find('span', class_ = 'housing') is not None:
                
                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    #make bedroom nan
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                    
                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                num_bedroom.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)
            #    num_bedroom.append(bedroom_count)
                
            #    sqft = np.nan
            #    sqfts.append(sqft)
        iterations += 1
        #print("Page" + str(iterations) + "Scraped Successfully!")   

Total number of search result: 3000
CPU times: total: 4.48 s
Wall time: 48.6 s


In [7]:
# create DataFrame
df = pd.DataFrame({'Post Datetime': post_date,
                   'Post Title': post_titles,
                   'Post URL': post_links,
                  'Neighborhood': post_cities,
                  'Bedroom': num_bedroom,
                  'SQFT': sqfts,
                  'Price': prices})

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3052 entries, 0 to 3051
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3052 non-null   object 
 1   Post Title     3052 non-null   object 
 2   Post URL       3052 non-null   object 
 3   Neighborhood   3052 non-null   object 
 4   Bedroom        2854 non-null   object 
 5   SQFT           2686 non-null   float64
 6   Price          3052 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 167.0+ KB
None


,Post Datetime,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
0,2022-12-09 13:36,One Bedroom & Den in Yaletown Park,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,537.0,2650
1,2022-12-09 13:35,One Bedroom & Den in Yaletown Park,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,537.0,2650
2,2022-12-09 13:35,"$2500/ 2br - LARGE 2 BEDROOM, 1 BATH WALKOUT B...",https://vancouver.craigslist.org/van/apa/d/por...,WESTWOOD PLATEAU COQUITLAM,2,NaN,2500
3,2022-12-09 13:34,Luxury Downtown Granville One bedroom,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,570.0,2400
4,2022-12-09 13:32,Awesome WATER & PARK Views @ PARK WEST ONE Rig...,https://vancouver.craigslist.org/van/apa/d/van...,2bd+2bth+den+flx+pkg YALETOWN FAVOURITE 455 Be...,2,1055.0,3800


In [8]:
df.tail()

,Post Datetime,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
3047,2022-11-20 08:51,Vancouver Yaletown Ocean view south facing new...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,3,1050.0,4380
3048,2022-11-20 02:13,1 bedroom w/central AC available mid Dec 2022,https://vancouver.craigslist.org/van/apa/d/van...,Nanaimo and 1st Ave,1,480.0,1650
3049,2022-11-20 01:36,"Steps from Main St, 2 BR/2 Bath condo with par...",https://vancouver.craigslist.org/van/apa/d/van...,Main & 18th,2,985.0,3750
3050,2022-11-19 22:31,5500sqft Whole house for rent I South Granville,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver west,7,5500.0,13000
3051,2022-11-19 22:07,Absolutely Beautiful Vancouver West 2-Bedroom ...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,1500.0,3600


## Process

Check the dataset I created from the Craigslist ad and final adjustment of the dataset to analyze the data. 

First to check the dataset:

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3052 entries, 0 to 3051
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3052 non-null   object 
 1   Post Title     3052 non-null   object 
 2   Post URL       3052 non-null   object 
 3   Neighborhood   3052 non-null   object 
 4   Bedroom        2854 non-null   object 
 5   SQFT           2686 non-null   float64
 6   Price          3052 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 167.0+ KB


Convert Post Datetime object to Datetime

I would like to distingish FULLY FURNISHED suite and not furnished suite. I will create another column to show if it isTURE or FALSE. I noticed the title says if it is furnished suite or not (If not on the title, I assume that the suite is not furnished) 

Also, I noticed that the number of bedroom is string, so I will convert to integer. 

In [10]:
# Convert Datetime then split seperate columns
df['Post Datetime'] = pd.to_datetime(df["Post Datetime"])
df['Post Date'] = df['Post Datetime'].dt.date
df['Post Time'] = df['Post Datetime'].dt.time
df = df[['Post Datetime', 'Post Date', 'Post Time', 'Post Title', 'Post URL', 'Neighborhood', 'Bedroom', 'SQFT', 'Price']]
df.tail()

,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
3047,2022-11-20 08:51:00,2022-11-20,08:51:00,Vancouver Yaletown Ocean view south facing new...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,3,1050.0,4380
3048,2022-11-20 02:13:00,2022-11-20,02:13:00,1 bedroom w/central AC available mid Dec 2022,https://vancouver.craigslist.org/van/apa/d/van...,Nanaimo and 1st Ave,1,480.0,1650
3049,2022-11-20 01:36:00,2022-11-20,01:36:00,"Steps from Main St, 2 BR/2 Bath condo with par...",https://vancouver.craigslist.org/van/apa/d/van...,Main & 18th,2,985.0,3750
3050,2022-11-19 22:31:00,2022-11-19,22:31:00,5500sqft Whole house for rent I South Granville,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver west,7,5500.0,13000
3051,2022-11-19 22:07:00,2022-11-19,22:07:00,Absolutely Beautiful Vancouver West 2-Bedroom ...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,1500.0,3600


In [11]:
# Convert Bedroom number strings to Integer then NaN value to 0 assume 0 bedroom is studio appartment. 
df["Bedroom"] = pd.to_numeric(df["Bedroom"], errors='ignore').astype('Int64')
df["Bedroom"] = df["Bedroom"].fillna(0)

In [12]:
%%time
# Check if word 'furnished' in the title post. 
search_word = 'furnished'
furnished = []
for i in range(len(df)):
    post_title_list = df['Post Title'][i].lower().split(" ")
    if any(word == search_word for word in post_title_list):
        furnished.append(True)
    else:
        furnished.append(False)
df["Furnished?"] = furnished

CPU times: total: 15.6 ms
Wall time: 14.5 ms


Check the number of $0 posts then do something with them based on the result. 

In [13]:
# How many ads are posted less than $50?
posted_low_price = df[df['Price'] < 50]
print(f'Number of $0 post: {len(posted_low_price)}')
posted_low_price

Number of $0 post: 0


,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?


In this case, only 13 total posts less than $50, so I will drop those rows. 

In [14]:
df.drop(index=posted_low_price.index, axis=0, inplace=True)
len(df)

3052

In [15]:
df.head(5)

,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?
0,2022-12-09 13:36:00,2022-12-09,13:36:00,One Bedroom & Den in Yaletown Park,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,537.0,2650,False
1,2022-12-09 13:35:00,2022-12-09,13:35:00,One Bedroom & Den in Yaletown Park,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,537.0,2650,False
2,2022-12-09 13:35:00,2022-12-09,13:35:00,"$2500/ 2br - LARGE 2 BEDROOM, 1 BATH WALKOUT B...",https://vancouver.craigslist.org/van/apa/d/por...,WESTWOOD PLATEAU COQUITLAM,2,NaN,2500,True
3,2022-12-09 13:34:00,2022-12-09,13:34:00,Luxury Downtown Granville One bedroom,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,570.0,2400,False
4,2022-12-09 13:32:00,2022-12-09,13:32:00,Awesome WATER & PARK Views @ PARK WEST ONE Rig...,https://vancouver.craigslist.org/van/apa/d/van...,2bd+2bth+den+flx+pkg YALETOWN FAVOURITE 455 Be...,2,1055.0,3800,False


In [17]:
# save to csv file
df.to_csv(f"C:\\Users\\Masa\\Desktop\\Chilliwack Real Estate\\Data\\{city}_rental_ads.csv", mode='a', header=False)

## Analyze

In [18]:
# Count how many rentals are furnished 
num_furnished_suite = df[df['Furnished?'] == True]["Furnished?"].count()
num_unfurnished_suite = len(df) - num_furnished_suite
furnished_suite_percent = np.round((num_furnished_suite / len(df)) * 100, 2)
print(f"Number of furnished suite is {num_furnished_suite}, and it's {furnished_suite_percent}% of the entire list.")

Number of furnished suite is 660, and it's 21.63% of the entire list.


In [19]:
# Find out the number of ads on each bedrooms number.  
num_bed = np.sort(df["Bedroom"].unique())
suite_counts = []
for i in range(len(num_bed)):
    suite_count = len(df[df['Bedroom'] == num_bed[i]])
    suite_counts.append(suite_count)
    #print(f'Number of {num_bed[i]} Bedroom suite: {len(df[df["Bedroom"] == num_bed[i]])}')
#print(f"List of suite counts: {suite_counts}")

In [20]:
# Find out Mean, Max, Min of SQFT and Price of each bedroom and SUM of furnished suite and total suite on each bedroom. 
df_count = df.groupby(by=['Bedroom']).agg({'SQFT': ['mean', 'max', 'min'],
                                                 'Price': ['mean', 'max', 'min'],
                                                  'Furnished?': ['sum', 'count']})
df_count['percent furnished'] = df_count['Furnished?']['sum'] / df_count["Furnished?"]['count']
df_count

SQFT                         Price               Furnished?  \
                mean      max     min         mean     max   min        sum   
Bedroom                                                                       
0         435.723164   1357.0   103.0  2244.994949    8000   800         59   
1         624.260967   4500.0     1.0  2519.943863   10000   100        266   
2         988.598963   4495.0   300.0  4289.877138  500000   700        258   
3        1563.080702   4000.0    14.0  5476.806250   24995  1300         43   
4        2556.875000   6800.0   900.0  6487.101010   25000   950         14   
5        3644.981481   6500.0  1800.0  8132.706897   19500  3300         13   
6        5191.260870  10098.0  1811.0  9009.000000   22880   950          5   
7        4117.750000   6000.0  1300.0  9898.888889   19995  1300          2   
8        4369.200000   4980.0  3106.0  8091.666667    9350  6500          0   

              percent furnished  
        count                    
Bedroom                          
0         198          0.297980  
1        1051          0.253092  
2        1286          0.200622  
3         320          0.134375  
4          99          0.141414  
5          58          0.224138  
6          25          0.200000  
7           9          0.222222  
8           6          0.000000

In [141]:
# Price per furnished suit on each bedroom
#df_price_furnished = df.drop(['Post Datetime', 'Post Title', 'Post URL', 'Neighborhood'], axis=1)
df_price_furnished = df.groupby(['Bedroom', 'Furnished?']).agg({'Price': ['mean', 'max', 'min', 'count'],
                                                               "SQFT": ['mean', 'max', 'min']})
df_price_furnished

Price                               SQFT           \
                            mean     max    min count          mean      max   
Bedroom Furnished?                                                             
0       False        2127.863636    4050    375    88    457.144737   1200.0   
        True         2245.740741    3000   1250    27    396.791667    550.0   
1       False        2233.338688    6500     90   747    627.744113   2000.0   
        True         2831.527273    6495    960   165    625.512821   1500.0   
2       False        3257.556391   28520    700  1064   1057.086458  85000.0   
        True         7267.075676  500000    140   185   1091.274854   2873.0   
3       False        4283.808625   19975    138   371   1546.578616   4256.0   
        True         7259.880952   24995   2400    42   1742.769231   3800.0   
4       False        4957.584337   23000    800   166   2522.865385   4900.0   
        True         7284.384615   12000   4200    13   2957.750000   4500.0   
5       False        7002.078947   20000   3100    76   3654.835616   7600.0   
        True        13474.444444   39990   6495     9   3940.222222   5271.0   
6       False        7436.052632   17000   3880    19   5694.684211  29090.0   
        True        22880.000000   22880  22880     1  10000.000000  10000.0   
7       False        6813.428571    9995   4800     7   4012.500000   5300.0   
        True        19995.000000   19995  19995     1   6000.000000   6000.0   
8       False        9375.000000   12000   5600     4   6484.000000   8084.0   
        True        49997.500000   59995  40000     2  16269.500000  16294.0   

                             
                        min  
Bedroom Furnished?           
0       False         228.0  
        True          228.0  
1       False         150.0  
        True          275.0  
2       False         300.0  
        True          550.0  
3       False         650.0  
        True          900.0  
4       False         900.0  
        True         1750.0  
5       False        1796.0  
        True         2500.0  
6       False        2250.0  
        True        10000.0  
7       False        2900.0  
        True         6000.0  
8       False        4788.0  
        True        16245.0

Dropped extrimely low price add, and it seems no outlier on the price

In [24]:
# Count how many ads per day. 
df.groupby('Post Date').agg({'Post Title': 'count'})

,Post Title
Post Date,
2022-11-16,1
2022-11-17,2
2022-11-19,5
2022-11-20,33
2022-11-21,39
2022-11-22,79
2022-11-23,65
2022-11-24,49
2022-11-25,46


It looks like around 1000 posts per day on weekday. 

### Create dataFrame for just one day

In [25]:
# December 7, 2022's ads
df_one_day = df[df['Post Date'] == datetime(2022, 12, 9).date()]
df_one_day

,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?
0,2022-12-09 13:36:00,2022-12-09,13:36:00,One Bedroom & Den in Yaletown Park,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,537.0,2650,False
1,2022-12-09 13:35:00,2022-12-09,13:35:00,One Bedroom & Den in Yaletown Park,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,537.0,2650,False
2,2022-12-09 13:35:00,2022-12-09,13:35:00,"$2500/ 2br - LARGE 2 BEDROOM, 1 BATH WALKOUT B...",https://vancouver.craigslist.org/van/apa/d/por...,WESTWOOD PLATEAU COQUITLAM,2,NaN,2500,True
3,2022-12-09 13:34:00,2022-12-09,13:34:00,Luxury Downtown Granville One bedroom,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,570.0,2400,False
4,2022-12-09 13:32:00,2022-12-09,13:32:00,Awesome WATER & PARK Views @ PARK WEST ONE Rig...,https://vancouver.craigslist.org/van/apa/d/van...,2bd+2bth+den+flx+pkg YALETOWN FAVOURITE 455 Be...,2,1055.0,3800,False
...,...,...,...,...,...,...,...,...,...,...
273,2022-12-09 00:15:00,2022-12-09,00:15:00,Kitsilano - 2BR available January 25th,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,NaN,2100,False
274,2022-12-09 00:14:00,2022-12-09,00:14:00,2 Br 2 Bath with den and flex room,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,1100.0,3600,False
275,2022-12-09 00:11:00,2022-12-09,00:11:00,Wonderful 2 bed/2 bath unit with storage and p...,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,2,815.0,3485,False
276,2022-12-09 00:02:00,2022-12-09,00:02:00,Gorgeous Ocean Views One Bedroom Plus Den Furn...,https://vancouver.craigslist.org/van/apa/d/van...,Coal Harbour,1,670.0,3350,True


## Share

## Act